# Webscraping

In [272]:

import pandas as pd
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin

df = pd.DataFrame(columns=["date", "text", "link"])
debatelink = []

base_url = "https://www.debates.org"
page = requests.get(urljoin(base_url, "/voter-education/debate-transcripts/"))
soup = BeautifulSoup(page.content, 'html.parser')

links = soup.find_all("a")
for link in links:
    if "debate-transcript" in link.get("href") and "Media" not in link.get("href") and "vice" not in link.get("href"):
        debatelink.append(urljoin(base_url, link.get("href")))
        df.loc[len(df)] = [None, None, urljoin(base_url, link.get("href"))]

for link in df["link"]:
    try:
        page = requests.get(link)
        soup = BeautifulSoup(page.content, 'html.parser')
        text = soup.find_all("p")
        text = [t.get_text() for t in text]
        text = " ".join(text)
        df.loc[df["link"] == link, "text"] = text

        date_link = link.split("/")[5:6] 
        date_link = "-".join(date_link[0].split("-")[:3])
        df.loc[df["link"] == link, "date"] = date_link
    except requests.exceptions.RequestException as e:
        print(f"Error accessing {link}: {e}")

df


,date,text,link
0,,Unofficial transcripts of most presidential an...,https://www.debates.org/voter-education/debate...
1,october-22-2020,Presidential Debate at Belmont University in N...,https://www.debates.org/voter-education/debate...
2,september-29-2020,Presidential Debate at Case Western Reserve Un...,https://www.debates.org/voter-education/debate...
3,october-19-2016,Presidential Debate at the University of Nevad...,https://www.debates.org/voter-education/debate...
4,october-9-2016,Presidential Debate at Washington University i...,https://www.debates.org/voter-education/debate...
...,...,...,...
88,october-22-1976,"\nOctober 22, 1976\n The Third Carter-Ford Pre...",https://www.debates.org/voter-education/debate...
89,september-26-1960,"\nSeptember 26, 1960\n The First Kennedy-Nixon...",https://www.debates.org/voter-education/debate...
90,october-7-1960,"\nOctober 7, 1960\n The Second Kennedy-Nixon P...",https://www.debates.org/voter-education/debate...
91,october-13-1960,"\nOctober 13, 1960\n The Third Kennedy-Nixon P...",https://www.debates.org/voter-education/debate...


In [273]:
len(debatelink)

93

In [274]:
# req = requests.get('https://www.presidency.ucsb.edu/documents/republican-candidates-debate-des-moines-iowa-1')

In [275]:
# req = requests.get('https://www.presidency.ucsb.edu/documents/republican-candidates-debate-des-moines-iowa-1')

In [276]:
# req.status_code

In [277]:
# req.content

In [278]:
# req.text

In [279]:
# sitesoup = BeautifulSoup(req.content)

In [280]:
# [tag.text for tag in sitesoup.select('p')]

In [281]:
import re 
# Function to extract text between all-caps words followed by a colon
def extract_text_segments(text, name):
    pattern = f"{name}:\s*([\s\S]*?)(?=(?:[A-Z]+\s*:\s*|$))"
    matches = re.finditer(pattern, text)
    indices = [(match.start(), match.end()) for match in matches]

    extracted_texts = []
    for i in range(len(indices) - 1):
        start = indices[i][0]
        end = indices[i][1]
        extracted_text = text[start:end].strip()
        extracted_texts.append(extracted_text)

    return extracted_texts

extract_text_segments(df['text'][5], 'CLINTON')

[]

loop through dftext get just clinton, add that extracted text to a row 'clinton' in new dataframe
- year = 
- dict = {'Year':None, 
        'Name':None 
        'Text': None
- df2 = pd.DataFrame(dict) 
- df2.loc[len(df.index)] = ['year, 'name', extracted text]  
- split last 4 characters of 'date' value in df

In [282]:
# names = ['CLINTON', 'TRUMP', 'OBAMA', 'ROMNEY',
#         'MCCAIN', 'BUSH', 'KERRY', 'GORE', 'CLINTON', 'DOLE',
#         'DUKAKIS', 'REAGAN', 'MONDALE', 'CARTER', 'FORD', 'NIXON',
#         'KENNEDY', 'ANDERSON']

# dict = {'Year':[None], 
#         'Name':[None], 
#         'Text': [None]}
# df2 = pd.DataFrame(dict) 
                            
# def text_extractor(data):
#     for row in data:
#         for name in names:
#             extracted_text = extract_text_segments(data, name)
#             year = df['date'].str[-4:]
#             df2.loc[len(df.index)] = [year, name, extracted_text]
#     return df2

# text_extractor(df['text'])

In [283]:
import pandas as pd

names = ['CLINTON', 'TRUMP', 'OBAMA', 'ROMNEY', 'MCCAIN', 'BUSH', 'KERRY', 'GORE', 'CLINTON', 'DOLE', 'DUKAKIS', 'REAGAN', 'MONDALE', 'CARTER', 'FORD', 'NIXON', 'KENNEDY', 'ANDERSON']

dict2 = {'Year': [], 'Name': [], 'Text': []}
df2 = pd.DataFrame(dict2)

def text_extractor(data):
    for name in names:
        for index, row in df.iterrows():
            if name in row['text']:
                extracted_text = extract_text_segments(row['text'], name)
                year = row['date'][-4:]
                df2.loc[len(df2.index)] = [year, name, extracted_text]
    return df2

result_df = text_extractor(df)
print(result_df)


/Users/natehiatt/anaconda3/envs/learn-env/lib/python3.9/site-packages/pandas/core/dtypes/cast.py:950: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  element = np.asarray(element)


     Year      Name                                               Text
0    2016   CLINTON  [CLINTON: Thank you very much, Chris. And than...
1    2016   CLINTON  [CLINTON: Well, thank you. Are you a teacher? ...
2    2016   CLINTON  [CLINTON: How are you, Donald? [applause], CLI...
3    1996   CLINTON  [CLINTON: I was going to applaud, too. Well, t...
4    1996   CLINTON  [CLINTON: Thank you, Jim. And thank you to the...
..    ...       ...                                                ...
163  1960   KENNEDY  [KENNEDY: In the first place I’ve never sugges...
164  1960   KENNEDY  [KENNEDY: Good evening, Mr. Shadel. MR., KENNE...
165  1960   KENNEDY  [KENNEDY: Good evening, Mr. Howe. MR., KENNEDY...
166  1980  ANDERSON  [ANDERSON: Miss Loomis, I think it’s very appr...
167  1980  ANDERSON  [ANDERSON: Miss Loomis, I think it’s very appr...

[168 rows x 3 columns]


In [284]:
result_df.to_csv('test.csv')

In [285]:
result_df.iloc[166] == result_df.iloc[167]

Year    True
Name    True
Text    True
dtype: bool

In [286]:
result_df['Text'] = result_df['Text'].drop_duplicates(keep='first')
result_df

,Year,Name,Text
0,2016,CLINTON,"[CLINTON: Thank you very much, Chris. And than..."
1,2016,CLINTON,"[CLINTON: Well, thank you. Are you a teacher? ..."
2,2016,CLINTON,"[CLINTON: How are you, Donald? [applause], CLI..."
3,1996,CLINTON,"[CLINTON: I was going to applaud, too. Well, t..."
4,1996,CLINTON,"[CLINTON: Thank you, Jim. And thank you to the..."
...,...,...,...
163,1960,KENNEDY,NaN
164,1960,KENNEDY,NaN
165,1960,KENNEDY,NaN
166,1980,ANDERSON,"[ANDERSON: Miss Loomis, I think it’s very appr..."


In [287]:
result_df.dropna(inplace=True)

In [288]:
result_df[result_df['Text'].isna() == True]

,Year,Name,Text


In [289]:
result_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 74 entries, 0 to 166
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Year    74 non-null     object
 1   Name    74 non-null     object
 2   Text    74 non-null     object
dtypes: object(3)
memory usage: 2.3+ KB


In [290]:
mask = (result_df['Name'] == "CLINTON") & (result_df['Year'] == "2016")
result_df.loc[mask, 'Name'] = "H_CLINTON"

In [291]:
mask = (result_df['Name'] == "CLINTON") & (result_df['Year'] == "1992")
result_df.loc[mask, 'Name'] = "B_CLINTON"

In [292]:
mask = (result_df['Name'] == "CLINTON") & (result_df['Year'] == "1996")
result_df.loc[mask, 'Name'] = "B_CLINTON"

In [293]:
mask = (result_df['Name'] == "BUSH") & (result_df['Year'] == "2008")
result_df.loc[mask, 'Name'] = "BUSH_2"

In [294]:
mask = (result_df['Name'] == "BUSH") & (result_df['Year'] == "2004")
result_df.loc[mask, 'Name'] = "BUSH_2"

In [295]:
mask = (result_df['Name'] == "BUSH") & (result_df['Year'] == "2000")
result_df.loc[mask, 'Name'] = "BUSH_2"

In [296]:
mask = (result_df['Name'] == "BUSH") & (result_df['Year'] == "1988") 
result_df.loc[mask, 'Name'] = "BUSH_1"

In [297]:
mask = (result_df['Name'] == "BUSH") & (result_df['Year'] == "1992") 
result_df.loc[mask, 'Name'] = "BUSH_1"

In [298]:
result_df['Name'].value_counts()

Name
B_CLINTON    7
BUSH_1       7
OBAMA        6
BUSH_2       6
GORE         5
TRUMP        5
KENNEDY      4
NIXON        4
FORD         4
CARTER       4
H_CLINTON    3
KERRY        3
MCCAIN       3
ROMNEY       3
DOLE         2
DUKAKIS      2
REAGAN       2
MONDALE      2
BUSH         1
ANDERSON     1
Name: count, dtype: int64

In [299]:
result_df = result_df[result_df['Name'] != "BUSH"]

In [300]:
result_df

,Year,Name,Text
0,2016,H_CLINTON,"[CLINTON: Thank you very much, Chris. And than..."
1,2016,H_CLINTON,"[CLINTON: Well, thank you. Are you a teacher? ..."
2,2016,H_CLINTON,"[CLINTON: How are you, Donald? [applause], CLI..."
3,1996,B_CLINTON,"[CLINTON: I was going to applaud, too. Well, t..."
4,1996,B_CLINTON,"[CLINTON: Thank you, Jim. And thank you to the..."
...,...,...,...
158,1960,KENNEDY,"[KENNEDY: Good evening, Mr. Howe. MR., KENNEDY..."
159,1960,KENNEDY,"[KENNEDY: Good evening, Mr. Shadel. MR., KENNE..."
160,1960,KENNEDY,[KENNEDY: In the first place I’ve never sugges...
161,1960,KENNEDY,"[KENNEDY: Mr. Smith, Mr. Nixon. In the electio..."


In [301]:
len(result_df)

73

In [302]:
result_df['Name'].value_counts()

Name
BUSH_1       7
B_CLINTON    7
OBAMA        6
BUSH_2       6
GORE         5
TRUMP        5
CARTER       4
KENNEDY      4
NIXON        4
FORD         4
H_CLINTON    3
KERRY        3
MCCAIN       3
ROMNEY       3
DUKAKIS      2
REAGAN       2
MONDALE      2
DOLE         2
ANDERSON     1
Name: count, dtype: int64

In [303]:
dems = ['B_CLINTON', 'OBAMA', 'GORE', 'CARTER', 'KENNEDY', 'H_CLINTON', 'KERRY', 'DUKAKIS', 'MONDALE']
ind = ['DOLE', 'ANDERSON']

result_df['Target'] = 2  # Default value for rows not matching dems or ind

for row in result_df['Name']:
    if row in dems:
        result_df.loc[result_df['Name'] == row, 'Target'] = 0
    elif row in ind:
        result_df.loc[result_df['Name'] == row, 'Target'] = 1

result_df


/var/folders/gf/gqtw2hsd3xx5_9tt2z47xflr0000gn/T/ipykernel_35357/569863851.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  result_df['Target'] = 2  # Default value for rows not matching dems or ind


,Year,Name,Text,Target
0,2016,H_CLINTON,"[CLINTON: Thank you very much, Chris. And than...",0
1,2016,H_CLINTON,"[CLINTON: Well, thank you. Are you a teacher? ...",0
2,2016,H_CLINTON,"[CLINTON: How are you, Donald? [applause], CLI...",0
3,1996,B_CLINTON,"[CLINTON: I was going to applaud, too. Well, t...",0
4,1996,B_CLINTON,"[CLINTON: Thank you, Jim. And thank you to the...",0
...,...,...,...,...
158,1960,KENNEDY,"[KENNEDY: Good evening, Mr. Howe. MR., KENNEDY...",0
159,1960,KENNEDY,"[KENNEDY: Good evening, Mr. Shadel. MR., KENNE...",0
160,1960,KENNEDY,[KENNEDY: In the first place I’ve never sugges...,0
161,1960,KENNEDY,"[KENNEDY: Mr. Smith, Mr. Nixon. In the electio...",0


In [304]:
result_df[result_df['Name'] == "REAGAN"]

,Year,Name,Text,Target
126,1980,REAGAN,[REAGAN: I don’t know what the differences mig...,2
127,1980,REAGAN,[REAGAN: I believe that the only unpopular mea...,2


In [305]:
result_df['Target'].value_counts()

Target
0    36
2    34
1     3
Name: count, dtype: int64

# NLP

In [306]:
from collections import defaultdict
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

import nltk
nltk.download('wordnet')
nltk.download('tagsets')
nltk.download('averaged_perceptron_tagger')
nltk.download('stopwords')
from nltk.probability import FreqDist
from nltk.corpus import stopwords
from nltk.tokenize import regexp_tokenize, word_tokenize, RegexpTokenizer
from nltk import pos_tag
from nltk.corpus import stopwords, wordnet
from nltk.stem import WordNetLemmatizer
import matplotlib.pyplot as plt
import string
import re
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, precision_score, ConfusionMatrixDisplay

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer,\
HashingVectorizer

[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/natehiatt/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package tagsets to
[nltk_data]     /Users/natehiatt/nltk_data...
[nltk_data]   Package tagsets is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/natehiatt/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/natehiatt/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [320]:
result_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 73 entries, 0 to 166
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Year        73 non-null     object
 1   Name        73 non-null     object
 2   Text        73 non-null     object
 3   Target      73 non-null     int64 
 4   text_lower  73 non-null     object
dtypes: int64(1), object(4)
memory usage: 3.4+ KB


In [339]:
result_df['text_lower'] = ["".join(item for item in lst).lower() for lst in result_df['Text']]

result_df

/var/folders/gf/gqtw2hsd3xx5_9tt2z47xflr0000gn/T/ipykernel_35357/1050904620.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  result_df['text_lower'] = ["".join(item for item in lst).lower() for lst in result_df['Text']]


,Year,Name,Text,Target,text_lower
0,2016,H_CLINTON,"[CLINTON: Thank you very much, Chris. And than...",0,"clinton: thank you very much, chris. and thank..."
1,2016,H_CLINTON,"[CLINTON: Well, thank you. Are you a teacher? ...",0,"clinton: well, thank you. are you a teacher? y..."
2,2016,H_CLINTON,"[CLINTON: How are you, Donald? [applause], CLI...",0,"clinton: how are you, donald? [applause]clinto..."
3,1996,B_CLINTON,"[CLINTON: I was going to applaud, too. Well, t...",0,"clinton: i was going to applaud, too. well, th..."
4,1996,B_CLINTON,"[CLINTON: Thank you, Jim. And thank you to the...",0,"clinton: thank you, jim. and thank you to the ..."
...,...,...,...,...,...
158,1960,KENNEDY,"[KENNEDY: Good evening, Mr. Howe. MR., KENNEDY...",0,"kennedy: good evening, mr. howe. mr.kennedy: m..."
159,1960,KENNEDY,"[KENNEDY: Good evening, Mr. Shadel. MR., KENNE...",0,"kennedy: good evening, mr. shadel. mr.kennedy:..."
160,1960,KENNEDY,[KENNEDY: In the first place I’ve never sugges...,0,kennedy: in the first place i’ve never suggest...
161,1960,KENNEDY,"[KENNEDY: Mr. Smith, Mr. Nixon. In the electio...",0,"kennedy: mr. smith, mr. nixon. in the election..."


In [355]:
result_df['text_lower'].isna().sum()

0

In [378]:
from nltk.corpus import stopwords
tokenizer = RegexpTokenizer(r"(?u)\w{3,}")
stopwords = stopwords.words("english")
lemmatizer = WordNetLemmatizer()

def preprocessing(text, tokenizer, stopwords, lemmatizer):
    # Make everything in the df["Text"] column into a lower-case string
    #text = ["".join(item for item in lst).lower() for lst in text]

    # Tokenize
    tokens = tokenizer.tokenize(text)
    
    # Remove stopwords
    tokens = [token for token in tokens if token not in stopwords]
    
    # Lemmatize
    tokens = [lemmatizer.lemmatize(token) for token in tokens]
    
    
    return tokens

In [380]:

# Apply the preprocessing function to the 'Text' column
result_df['tokens'] = result_df['text_lower'].apply(lambda x: preprocessing(x, tokenizer, stopwords, lemmatizer))
result_df


/var/folders/gf/gqtw2hsd3xx5_9tt2z47xflr0000gn/T/ipykernel_35357/1465720170.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  result_df['tokens'] = result_df['text_lower'].apply(lambda x: preprocessing(x, tokenizer, stopwords, lemmatizer))


,Year,Name,Text,Target,text_lower,tokens
0,2016,H_CLINTON,"[CLINTON: Thank you very much, Chris. And than...",0,"clinton: thank you very much, chris. and thank...","[clinton, thank, much, chris, thanks, unlv, ho..."
1,2016,H_CLINTON,"[CLINTON: Well, thank you. Are you a teacher? ...",0,"clinton: well, thank you. are you a teacher? y...","[clinton, well, thank, teacher, yes, think, go..."
2,2016,H_CLINTON,"[CLINTON: How are you, Donald? [applause], CLI...",0,"clinton: how are you, donald? [applause]clinto...","[clinton, donald, applause, clinton, well, tha..."
3,1996,B_CLINTON,"[CLINTON: I was going to applaud, too. Well, t...",0,"clinton: i was going to applaud, too. well, th...","[clinton, going, applaud, well, thank, jim, th..."
4,1996,B_CLINTON,"[CLINTON: Thank you, Jim. And thank you to the...",0,"clinton: thank you, jim. and thank you to the ...","[clinton, thank, jim, thank, people, hartford,..."
...,...,...,...,...,...,...
158,1960,KENNEDY,"[KENNEDY: Good evening, Mr. Howe. MR., KENNEDY...",0,"kennedy: good evening, mr. howe. mr.kennedy: m...","[kennedy, good, evening, howe, kennedy, howe, ..."
159,1960,KENNEDY,"[KENNEDY: Good evening, Mr. Shadel. MR., KENNE...",0,"kennedy: good evening, mr. shadel. mr.kennedy:...","[kennedy, good, evening, shadel, kennedy, mcge..."
160,1960,KENNEDY,[KENNEDY: In the first place I’ve never sugges...,0,kennedy: in the first place i’ve never suggest...,"[kennedy, first, place, never, suggested, cuba..."
161,1960,KENNEDY,"[KENNEDY: Mr. Smith, Mr. Nixon. In the electio...",0,"kennedy: mr. smith, mr. nixon. in the election...","[kennedy, smith, nixon, election, 1860, abraha..."


In [383]:
result_df['tokens'].isna().sum()

0

## NLP Modelling

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(df["string_tokens"], df['target'], random_state=42)

KeyError: 'target'

In [ ]:
vectorizer = TfidfVectorizer()
vectorizr.fit(X_train
X_train_vectorized = vectorizer.transform(X_train)
X_test_vectorized = vectorizer.transform(X_test)

In [ ]:
mnb = MultinomialNB()
mnb = mnb.fit(X_train_vectorized, y_train)


In [ ]:
import matplotlib.pyplot as plt
from nltk.probability import FreqDist

# Create a frequency distribution for each speaker
speaker_freq_dist = {}
for speaker in result_df['Name'].unique():
    tokens = [token for sublist in result_df[result_df['Name'] == speaker]['tokens'] for token in sublist]
    freq_dist = FreqDist(tokens)
    speaker_freq_dist[speaker] = freq_dist

# Plot the frequency distribution for each speaker
for speaker, freq_dist in speaker_freq_dist.items():
    plt.figure(figsize=(10, 6))
    freq_dist.plot(30, cumulative=False)
    plt.title(f"Word Usage Frequency Distribution for {speaker}")
    plt.xlabel("Words")
    plt.ylabel("Frequency")
    plt.show()
